In [ ]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
 
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open("drive/My Drive/Colab Notebooks/ChAI/intents.json").read()
intents = json.loads(data_file)

In [ ]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        # take each word and tokenize it
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        # adding documents
        documents.append((w, intent['tag'], intent["context"]))

        # adding classes to our class list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print (len(documents), "documents")

print (len(classes), "classes", classes)

print (len(words), "unique lemmatized words", words)


pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

47 documents
17 classes ['1', '10', '11', '12', '13', '14', '15', '16', '17', '2', '3', '4', '5', '6', '7', '8', '9']
88 unique lemmatized words ['19', 'a', 'about', 'alive', 'am', 'anyone', 'are', 'bad', 'be', 'bored', 'bot', 'can', 'corona', 'coronavirus', 'covid', 'covid-19', 'created', 'creator', 'damn', 'dark', 'day', 'did', 'do', 'donald', 'even', 'father', 'feeling', 'fun', 'funny', 'good', 'greatest', 'hear', 'hello', 'hey', 'hi', 'hitler', 'hola', 'how', 'i', 'india', 'is', 'joke', 'laugh', 'lmao', 'lol', 'love', 'man', 'me', 'minister', "n't", 'na', 'not', 'nothing', 'of', 'or', 'party', 'political', 'president', 'prime', 'should', 'so', 'something', 'state', 'sup', 'support', 'tell', 'that', 'the', 'there', 'think', 'trump', 'u', 'u.s', 'united', 'virus', 'wa', 'wan', 'wassup', 'we', 'what', 'whatsup', 'whatup', 'which', 'who', 'why', 'wrong', 'you', 'your']


In [ ]:
# initializing training data
training = []
output_empty = [0] * len(classes)
for doc in documents:
    # initializing bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    bag.append(int(doc[2]))
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


In [ ]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)
#pickle.dump(hist, open('model.pkl','wb'))

print("model created")

Epoch 1/200
10/10 [==============================] - 0s 1ms/step - loss: 3.0275 - accuracy: 0.0851
Epoch 2/200
10/10 [==============================] - 0s 1ms/step - loss: 2.8792 - accuracy: 0.1702
Epoch 3/200
10/10 [==============================] - 0s 2ms/step - loss: 2.7714 - accuracy: 0.1277
Epoch 4/200
10/10 [==============================] - 0s 2ms/step - loss: 2.7182 - accuracy: 0.2340
Epoch 5/200
10/10 [==============================] - 0s 2ms/step - loss: 2.7463 - accuracy: 0.2553
Epoch 6/200
10/10 [==============================] - 0s 1ms/step - loss: 2.6063 - accuracy: 0.1702
Epoch 7/200
10/10 [==============================] - 0s 2ms/step - loss: 2.3857 - accuracy: 0.3404
Epoch 8/200
10/10 [==============================] - 0s 1ms/step - loss: 2.3630 - accuracy: 0.2553
Epoch 9/200
10/10 [==============================] - 0s 1ms/step - loss: 2.1297 - accuracy: 0.3830
Epoch 10/200
10/10 [==============================] - 0s 1ms/step - loss: 2.3764 - accuracy: 0.2979
Epoch 11/

In [ ]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

def bow(sentence, words, context, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    bag.append(int(context))
    return(np.array(bag))



In [ ]:
sentence="why"
context="20"
p = bow(sentence, words, context, show_details=False)
res = model.predict(np.array([p]))[0]
ERROR_THRESHOLD = 0.25
results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
return_list = []
for r in results:
  return_list.append({"intent": classes[r[0]], "probability": str(r[1])})

tag = return_list[0]['intent']
list_of_intents = intents['intents']
for i in list_of_intents:
  if(i['tag']== tag):
    result = random.choice(i['responses'])
    break

result

'No reason...'

In [ ]:
0